In [1]:
import os
import pandas as pd
import numpy as np

Potential features: 

- Aircraft typecode
- Mean descent rate of the STAR
- Runway length
- Runway orientation
- obstructions near airport (boolean to indicate if there are mountains / buildings)
- Runway configuration (parallel, single)
- Weather visibility (fog / clear)
- Wind speed
- Wind direction (relative to the runway maybe, or something like that)
- Storm presence
- Traffic congestion (like number of aircraft in-flight at the same time)
- STAR Type (ILS VS RNAV)
- STAR Complexity (like number of waypoints or number of turns)
- Initial length of STAR
- Time of the day (night VS day)
- Season

****
#### EDDM
****

In [2]:
file_path = "C:\\Users\\kruu\\store\\"

data_EDDM = pd.read_parquet(os.path.join(file_path + "\\data_EDDM\\landing_df_EDDM.parquet"))

In [3]:
#Typecode selection

typecode_list = data_EDDM.typecode.value_counts()[data_EDDM.typecode.value_counts() > 400].index.tolist()
data_EDDM_reduced = data_EDDM.query(f"typecode in {typecode_list}")

print(f"typecode proportion: {len(typecode_list) / data_EDDM.typecode.nunique()}")
print(f"Flight proportion: {len(data_EDDM_reduced) / len(data_EDDM)} ")

typecode proportion: 0.07692307692307693
Flight proportion: 0.816451424200153 


In [4]:
#Flight airborne

start_matrix = data_EDDM_reduced['start'].values[:, np.newaxis]
stop_matrix = data_EDDM_reduced['stop'].values[:, np.newaxis]

# Find overlaps: (start1 <= stop2) & (stop1 >= start2)
overlap_matrix = (start_matrix <= stop_matrix.T) & (stop_matrix >= start_matrix.T)

# Count the number of overlaps for each row
data_EDDM_reduced['nb_aircraft'] = overlap_matrix.sum(axis=1)

C:\Users\kruu\AppData\Local\Temp\ipykernel_49116\1444051488.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_EDDM_reduced['nb_aircraft'] = overlap_matrix.sum(axis=1)


In [5]:
# aircraft bodytype

ac_body = {
    "Widebody": ["B763", "A333"],
    "Narrowbody": ["A320", "BCS3", "A20N", "A319", "BCS1", "A21N", "B738", "A321"],
    "Regional Jet": ["E190", "CRJ9", "E195", "DH8D"],
    "Business jets": ["PC12", "C56X", "F2TH", "PC24", "C68A", "E55P"],
}

def body_type(typecode, ac_body_dict):
    for key, value in ac_body_dict.items():
        if typecode in value:
            return key
    return None

data_EDDM_reduced[["body_type"]] =  data_EDDM_reduced.typecode.apply(lambda x: pd.Series(body_type(x, ac_body)))

C:\Users\kruu\AppData\Local\Temp\ipykernel_49116\195999770.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_EDDM_reduced[["body_type"]] =  data_EDDM_reduced.typecode.apply(lambda x: pd.Series(body_type(x, ac_body)))


In [6]:
# Time data
import pytz

data_EDDM_reduced["month"] = pd.DatetimeIndex(data_EDDM_reduced.start).month.astype('category')
data_EDDM_reduced["hour"] = pd.DatetimeIndex(data_EDDM_reduced.start).hour.astype('category')
data_EDDM_reduced["day"] = pd.DatetimeIndex(data_EDDM_reduced.start).day.astype('category')
data_EDDM_reduced["weekday"] = pd.DatetimeIndex(data_EDDM_reduced.start).weekday.astype('category')

def get_season(date):
    # Extract month and day
    month = date.month
    day = date.day
    
    # Define the seasons based on month and day
    if (month == 12 and day >= 21) or (month in [1, 2]) or (month == 3 and day < 20):
        return 'Winter'
    elif (month == 3 and day >= 20) or (month in [4, 5]) or (month == 6 and day < 21):
        return 'Spring'
    elif (month == 6 and day >= 21) or (month in [7, 8]) or (month == 9 and day < 23):
        return 'Summer'
    elif (month == 9 and day >= 23) or (month in [10, 11]) or (month == 12 and day < 21):
        return 'Fall'
    
data_EDDM_reduced['season'] = data_EDDM_reduced['start'].apply(get_season)

def is_rush_hour(date): #based on hourly count bar plot: data_EDDM_reduced.groupby("hour").id.count().plot(kind="bar")
    munich_tz = pytz.timezone('Europe/Munich')
    
    # Extract hour
    hour = date.tz_convert(munich_tz).hour
    minute = date.tz_convert(munich_tz).minute
    time_in_minutes = hour * 60 + minute
    
    if (5 * 60 <= time_in_minutes <= 6 * 60) or \
       (7 * 60 <= time_in_minutes <= 9 * 60) or \
       (11 * 60 <= time_in_minutes <= 14 * 60) or \
       (16 * 60 <= time_in_minutes <= 20 * 60):
        return True
    else:
        return False

data_EDDM_reduced['rush_hour'] = data_EDDM_reduced["start"].apply(is_rush_hour)

C:\Users\kruu\AppData\Local\Temp\ipykernel_49116\1971939380.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_EDDM_reduced["month"] = pd.DatetimeIndex(data_EDDM_reduced.start).month.astype('category')
C:\Users\kruu\AppData\Local\Temp\ipykernel_49116\1971939380.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_EDDM_reduced["hour"] = pd.DatetimeIndex(data_EDDM_reduced.start).hour.astype('category')
C:\Users\kruu\AppData\Local\Temp\ipykernel_49116\1971939380.py:6: SettingWithCopyWarning: 
A value

In [7]:
# Nominal distance proportion

star_len_eddm = {
    "NAPS1B": 61.9,
    "LAND1B": 74.8,
    "ROKI1A": 52.3,
    "BETO1A": 61.3,
}

data_EDDM_reduced["nominal_distance"] = data_EDDM_reduced.star.apply(lambda x: star_len_eddm[x])


C:\Users\kruu\AppData\Local\Temp\ipykernel_49116\1437668120.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_EDDM_reduced["nominal_distance"] = data_EDDM_reduced.star.apply(lambda x: star_len_eddm[x])


In [9]:
# Meteo Data

import requests
from requests.exceptions import RequestException, Timeout

import logging
import tqdm
import time
from traffic.data.weather import metar

file_path_save = "C:\\Users\\kruu\\store\\data_EDDM"

def get_meteo_data(row, airport):
    start = row.start
    stop = row.stop
    
    meteo = metar.METAR(airport).get(
        start = start,
        stop = stop,
    )
    
    def safe_mean(values):
        valid_values = [val.value() for val in values if val is not None]
        return np.mean(valid_values) if valid_values else None  # Return None if no valid values
    
    wind_dir = safe_mean(meteo.wind_dir.values)
    wind_speed = safe_mean(meteo.wind_speed.values)
    vis = safe_mean(meteo.vis.values) / 1000 # In km
    temp = safe_mean(meteo.temp.values)
    press = safe_mean(meteo.press.values) - 1013 # relative to standard pressure
    
    return wind_dir, wind_speed, vis, temp, press

def get_meteo_data_with_retry(row, airport, max_retries=3, delay=5):
    retries = 0
    while retries < max_retries:
        try:
            return get_meteo_data(row, airport)
        except RequestException as e:
            retries += 1
            print(f"Error: {e}, retrying ({retries}/{max_retries})...")
    return None, None, None, None, None  # If all retries fail, return None values

batch_size = 2000  # Smaller batch size
num_batches = len(data_EDDM_reduced) // batch_size + 1

for i in tqdm.tqdm(range(num_batches)):
    if not os.path.exists(os.path.join(file_path_save + f"landing_df_EDDM_with_meteo_{i}_of_{num_batches-1}.parquet")):
        batch = data_EDDM_reduced.iloc[i * batch_size:(i + 1) * batch_size]
        batch[["avg_wind_dir", "avg_wind_speed", "avg_vis", "avg_temp", "avg_press"]] = batch.apply(
            lambda row: pd.Series(get_meteo_data_with_retry(row, "EDDM")), axis=1
        )
        batch.to_parquet(os.path.join(file_path_save + f"landing_df_EDDM_with_meteo_{i}_of_{num_batches-1}.parquet"))
    else:
        print("file already exists")


  0%|          | 0/10 [00:00<?, ?it/s]

file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists


c:\Users\kruu\git_folder\STAR_shortcut_OSN_paper\.venv\Lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body '600 FG' while processing 'EDDM 260020Z AUTO VRB01KT 2200 0550 R08L/0600D R08R/1800N BCFG NCD 06/06 Q1024 TEMPO 600 FG'
  warnings.warn(message, RuntimeWarning)
c:\Users\kruu\git_folder\STAR_shortcut_OSN_paper\.venv\Lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body 'RE//' while processing 'EDDM 100850Z AUTO 27011KT 9999 FEW011 SCT015 OVC049 04/02 Q1022 RE// TEMPO BKN013'
  warnings.warn(message, RuntimeWarning)
c:\Users\kruu\git_folder\STAR_shortcut_OSN_paper\.venv\Lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body 'RE//' while processing 'EDDM 100850Z AUTO 27011KT 9999 FEW011 SCT015 OVC049 04/02 Q1022 RE// TEMPO BKN013'
  warnings.warn(message, RuntimeWarning)
c:\Users\kruu\git_folder\STAR_shortcut_OSN_paper\.venv\Lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body 'RE//' wh

****
### LIRF
****

In [2]:
file_path = "C:\\Users\\kruu\\store\\"

data_LIRF = pd.read_parquet(os.path.join(file_path + "\\data_LIRF\\landing_df_LIRF.parquet"))

In [3]:
#Typecode selection

typecode_list = data_LIRF.typecode.value_counts()[data_LIRF.typecode.value_counts() > 400].index.tolist()
data_LIRF_reduced = data_LIRF.query(f"typecode in {typecode_list}")

print(f"typecode proportion: {len(typecode_list) / data_LIRF.typecode.nunique()}")
print(f"Flight proportion: {len(data_LIRF_reduced) / len(data_LIRF)} ")

typecode proportion: 0.11827956989247312
Flight proportion: 0.8375257571634074 


In [4]:
#Flight airborne

start_matrix = data_LIRF_reduced['start'].values[:, np.newaxis]
stop_matrix = data_LIRF_reduced['stop'].values[:, np.newaxis]

# Find overlaps: (start1 <= stop2) & (stop1 >= start2)
overlap_matrix = (start_matrix <= stop_matrix.T) & (stop_matrix >= start_matrix.T)

# Count the number of overlaps for each row
data_LIRF_reduced['nb_aircraft'] = overlap_matrix.sum(axis=1)

C:\Users\kruu\AppData\Local\Temp\ipykernel_19880\1537632840.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_LIRF_reduced['nb_aircraft'] = overlap_matrix.sum(axis=1)


In [18]:
# aircraft bodytype

ac_body = {
    "Widebody": ["B763", "A333", "B789", "A332", "A333", "B772"],
    "Narrowbody": ["A320", "BCS3", "A20N", "A319", "BCS1", "A21N", "B738", "A321", "A20N", "A21N"],
    "Regional Jet": ["E190", "CRJ9", "E195", "DH8D"],
    "Business jets": ["PC12", "C56X", "F2TH", "PC24", "C68A", "E55P"],
}

def body_type(typecode, ac_body_dict):
    for key, value in ac_body_dict.items():
        if typecode in value:
            return key
    return None

data_LIRF_reduced[["body_type"]] =  data_LIRF_reduced.typecode.apply(lambda x: pd.Series(body_type(x, ac_body)))

C:\Users\kruu\AppData\Local\Temp\ipykernel_19880\1080642742.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_LIRF_reduced[["body_type"]] =  data_LIRF_reduced.typecode.apply(lambda x: pd.Series(body_type(x, ac_body)))


In [10]:
# Time data
import pytz

data_LIRF_reduced["month"] = pd.DatetimeIndex(data_LIRF_reduced.start).month.astype('category')
data_LIRF_reduced["hour"] = pd.DatetimeIndex(data_LIRF_reduced.start).hour.astype('category')
data_LIRF_reduced["day"] = pd.DatetimeIndex(data_LIRF_reduced.start).day.astype('category')
data_LIRF_reduced["weekday"] = pd.DatetimeIndex(data_LIRF_reduced.start).weekday.astype('category')

def get_season(date):
    # Extract month and day
    month = date.month
    day = date.day
    
    # Define the seasons based on month and day
    if (month == 12 and day >= 21) or (month in [1, 2]) or (month == 3 and day < 20):
        return 'Winter'
    elif (month == 3 and day >= 20) or (month in [4, 5]) or (month == 6 and day < 21):
        return 'Spring'
    elif (month == 6 and day >= 21) or (month in [7, 8]) or (month == 9 and day < 23):
        return 'Summer'
    elif (month == 9 and day >= 23) or (month in [10, 11]) or (month == 12 and day < 21):
        return 'Fall'
    
data_LIRF_reduced['season'] = data_LIRF_reduced['start'].apply(get_season)

def is_rush_hour(date):
    rome_tz = pytz.timezone('Europe/Rome')
    
    # Extract hour
    hour = date.tz_convert(rome_tz).hour
    minute = date.tz_convert(rome_tz).minute
    time_in_minutes = hour * 60 + minute
    
    if (6 * 60 + 30 <= time_in_minutes <= 9 * 60) or \
       (11 * 60 <= time_in_minutes <= 14 * 60) or \
       (16 * 60 + 30 <= time_in_minutes <= 18 * 60) or \
       (19 * 60 <= time_in_minutes <= 21 * 60):
        return True
    else:
        return False

data_LIRF_reduced['rush_hour'] = data_LIRF_reduced["start"].apply(is_rush_hour)

C:\Users\kruu\AppData\Local\Temp\ipykernel_19880\3397128224.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_LIRF_reduced["month"] = pd.DatetimeIndex(data_LIRF_reduced.start).month.astype('category')
C:\Users\kruu\AppData\Local\Temp\ipykernel_19880\3397128224.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_LIRF_reduced["hour"] = pd.DatetimeIndex(data_LIRF_reduced.start).hour.astype('category')
C:\Users\kruu\AppData\Local\Temp\ipykernel_19880\3397128224.py:6: SettingWithCopyWarning: 
A value

In [11]:
# Nominal distance proportion

star_len_eddm = {
    "ELKA2A": 132.7,
    "VALM2C": 92.7,
    "RITE2A": 94.8,
    "LAT2C": 74.3,
}

data_LIRF_reduced["nominal_distance"] = data_LIRF_reduced.star.apply(lambda x: star_len_eddm[x])

C:\Users\kruu\AppData\Local\Temp\ipykernel_19880\3240740452.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_LIRF_reduced["nominal_distance"] = data_LIRF_reduced.star.apply(lambda x: star_len_eddm[x])


In [36]:
# Meteo Data

import requests
from requests.exceptions import RequestException, Timeout

import logging
import tqdm
import time
from traffic.data.weather import metar

file_path_save = "C:\\Users\\kruu\\store\\data_LIRF\\"

def get_meteo_data(row, airport):
    start = row.start
    stop = row.stop
    
    try:
        # Attempt to retrieve the METAR data
        meteo = metar.METAR(airport).get(
            start=start,
            stop=stop,
        )
    except KeyError:
        print(f"KeyError for airport {airport}, start {start}, stop {stop}")
        return None, None, None, None, None
    except Exception as e:
        print(f"Error occurred: {e}")
        return None, None, None, None, None
    
    def safe_mean(values):
        valid_values = [val.value() for val in values if val is not None]
        return np.mean(valid_values) if valid_values else None  # Return None if no valid values
    
    wind_dir = safe_mean(meteo.wind_dir.values)
    wind_speed = safe_mean(meteo.wind_speed.values)
    vis = safe_mean(meteo.vis.values) / 1000 # In km
    temp = safe_mean(meteo.temp.values)
    press = safe_mean(meteo.press.values) - 1013 # relative to standard pressure
    
    return wind_dir, wind_speed, vis, temp, press


batch_size = 1000  # Smaller batch size
num_batches = len(data_LIRF_reduced) // batch_size + 1

for i in range(num_batches):
    if os.path.exists(os.path.join(file_path_save + f"landing_df_LIRF_with_meteo_{i}_of_{num_batches}.parquet")):
        print("file already exists")
    else:
        batch = data_LIRF_reduced.iloc[i * batch_size:(i + 1) * batch_size]
        batch[["avg_wind_dir", "avg_wind_speed", "avg_vis", "avg_temp", "avg_press"]] = batch.apply(
            lambda row: pd.Series(get_meteo_data(row, "LIRF")), axis=1
        )
        batch.to_parquet(os.path.join(file_path_save + f"landing_df_LIRF_with_meteo_{i}_of_{num_batches}.parquet"))
        
        


file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists


****
### LSGG
****


In [2]:
file_path = "C:\\Users\\kruu\\store\\"

data_LSGG = pd.read_parquet(os.path.join(file_path + "landing_df_LSGG.parquet"))
data_LSGG["airport"] = "LSGG"

#### Nominal STAR distance

In [3]:
data_LSGG = data_LSGG.query("typecode.isna() ==  False")
data_LSGG.runway = data_LSGG.runway.apply(lambda x: x[:2])

In [4]:
# STAR entry point + STAR IAF + runway + initial length in NM
star_len_lsgg = {
    "AKIT2N": ["AKITO", "INDIS", 95.2],
    "AKIT3R": ["AKITO", "SPR", 91.6],
    "BANK3N": ["BANKO", "INDIS", 67.2],
    "BANK3R": ["BANKO", "SPR", 63.6],
    "BELU3N": ["BELUS", "INDIS", 75.4],
    "BELU3R": ["BELUS", "SPR", 65.5],
    "BENO1N": ["BENOT", "INDIS", 90.1], # almost same distance as BENO1P / same entry + exit points
    "BENO1R": ["BENOT", "SPR", 47.5], # almost same distance as BENO1T / same entry + exit points
    "DJL2N": ["DJL", "INDIS", 94.9],
    "DJL2R": ["DJL", "SPR", 91.6],
    "KINE2N": ["KINES", "INDIS", 86.4],
    "KINE2R": ["KINES", "SPR", 82.8],
    "LUSA2N": ["LUSAR", "INDIS", 70.5],
    "LUSA2R": ["LUSAR", "SPR", 67.2],
    "ULME1N": ["ULMES", "INDIS", 90.7], # almost same distance as BENO1P / same entry + exit points
    "ULME1R": ["ULMES", "SPR", 45.5],
}


#APP len depending on IAF and runway
app_len_lsgg ={
    ("SPR", "22") : 18.7,
    ("INDIS", "04") : 17.4,
}

In [5]:
def find_star(row, star_dict, app_dict):
    navaid_iaf_tuple = (row['navaid'], row['iaf'])
    for key, value in star_dict.items():
        if tuple(value[:2]) == navaid_iaf_tuple:
            return key, value[2] + app_dict[(row['iaf'], row['runway'])]
    return None, None

data_LSGG["iaf"] = data_LSGG.runway.apply(lambda x: "SPR" if x[:2] == "22" else "INDIS")
data_LSGG[["star", "nominal_distance"]] =  data_LSGG.apply(lambda row: pd.Series(find_star(row, star_len_lsgg, app_len_lsgg)), axis = 1)

#### Traffic congestion

In [6]:
start_matrix = data_LSGG['start'].values[:, np.newaxis]
stop_matrix = data_LSGG['stop'].values[:, np.newaxis]

# Find overlaps: (start1 <= stop2) & (stop1 >= start2)
overlap_matrix = (start_matrix <= stop_matrix.T) & (stop_matrix >= start_matrix.T)

# Count the number of overlaps for each row
data_LSGG['nb_aircraft'] = overlap_matrix.sum(axis=1)

#### Aircraft type

In [7]:
typecode_list = data_LSGG.typecode.value_counts()[data_LSGG.typecode.value_counts() > 400].index.tolist()
data_LSGG_reduced = data_LSGG.query(f"typecode in {typecode_list}")

print(f"typecode proportion: {len(typecode_list) / data_LSGG.typecode.nunique()}")
print(f"Flight proportion: {len(data_LSGG_reduced) / len(data_LSGG)} ")

typecode proportion: 0.12195121951219512
Flight proportion: 0.8361164076192678 


In [34]:
# Weight category for the typecodes that appear more than 400 times per year

wtc_lsgg = {
    "H": ["B763", "A333"],
    "J": [],
    "L": ["PC12", ],
    "L/M": [],
    "M": ["A320", "BCS3", "A20N", "A319", "BCS1", "E190", "B738", "CRJ9", "E55P", "A21N", "E195", "C56X", "A321", "F2TH", "PC24", "DH8D", "C68A"],
}

#### Time data

In [8]:
# Raw time data

data_LSGG_reduced["month"] = pd.DatetimeIndex(data_LSGG_reduced.start).month.astype('category')
data_LSGG_reduced["hour"] = pd.DatetimeIndex(data_LSGG_reduced.start).hour.astype('category')
data_LSGG_reduced["day"] = pd.DatetimeIndex(data_LSGG_reduced.start).day.astype('category')
data_LSGG_reduced["weekday"] = pd.DatetimeIndex(data_LSGG_reduced.start).weekday.astype('category')

C:\Users\kruu\AppData\Local\Temp\ipykernel_14244\3953558574.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_LSGG_reduced["month"] = pd.DatetimeIndex(data_LSGG_reduced.start).month.astype('category')
C:\Users\kruu\AppData\Local\Temp\ipykernel_14244\3953558574.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_LSGG_reduced["hour"] = pd.DatetimeIndex(data_LSGG_reduced.start).hour.astype('category')
C:\Users\kruu\AppData\Local\Temp\ipykernel_14244\3953558574.py:5: SettingWithCopyWarning: 
A value

In [9]:
# Function to determine the season based on the date

def get_season(date):
    # Extract month and day
    month = date.month
    day = date.day
    
    # Define the seasons based on month and day
    if (month == 12 and day >= 21) or (month in [1, 2]) or (month == 3 and day < 20):
        return 'Winter'
    elif (month == 3 and day >= 20) or (month in [4, 5]) or (month == 6 and day < 21):
        return 'Spring'
    elif (month == 6 and day >= 21) or (month in [7, 8]) or (month == 9 and day < 23):
        return 'Summer'
    elif (month == 9 and day >= 23) or (month in [10, 11]) or (month == 12 and day < 21):
        return 'Fall'
    
data_LSGG_reduced['season'] = data_LSGG_reduced['start'].apply(get_season)

C:\Users\kruu\AppData\Local\Temp\ipykernel_14244\4075467397.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_LSGG_reduced['season'] = data_LSGG_reduced['start'].apply(get_season)


In [10]:
import pytz

# Rush hour (I've taken Rome rush hours here, maybe it's different in Geneva)

def is_rush_hour(date):
    
    geneva_tz = pytz.timezone('Europe/Zurich')
    
    # Extract hour
    hour = date.tz_convert(geneva_tz).hour
    minute = date.tz_convert(geneva_tz).minute
    time_in_minutes = hour * 60 + minute
    
    if (6 * 60 + 30 <= time_in_minutes <= 9 * 60) or \
       (11 * 60 <= time_in_minutes <= 14 * 60) or \
       (16 * 60 + 30 <= time_in_minutes <= 18 * 60) or \
       (19 * 60 <= time_in_minutes <= 21 * 60):
        return True
    else:
        return False

data_LSGG_reduced['rush_hour'] = data_LSGG_reduced["start"].apply(is_rush_hour)

C:\Users\kruu\AppData\Local\Temp\ipykernel_14244\3789480556.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_LSGG_reduced['rush_hour'] = data_LSGG_reduced["start"].apply(is_rush_hour)



#### Meteo Data


In [11]:
import requests
from requests.exceptions import RequestException, Timeout

import logging
import tqdm
import time
from traffic.data.weather import metar

file_path_save = "C:\\Users\\kruu\\store\\"

def get_meteo_data(row, airport):
    start = row.start
    stop = row.stop
    
    meteo = metar.METAR(airport).get(
        start = start,
        stop = stop,
    )
    
    def safe_mean(values):
        valid_values = [val.value() for val in values if val is not None]
        return np.mean(valid_values) if valid_values else None  # Return None if no valid values
    
    wind_dir = safe_mean(meteo.wind_dir.values)
    wind_speed = safe_mean(meteo.wind_speed.values)
    vis = safe_mean(meteo.vis.values) / 1000 # In km
    temp = safe_mean(meteo.temp.values)
    press = safe_mean(meteo.press.values) - 1013 # relative to standard pressure
    
    return wind_dir, wind_speed, vis, temp, press

def get_meteo_data_with_retry(row, airport, max_retries=3, delay=5):
    retries = 0
    while retries < max_retries:
        try:
            return get_meteo_data(row, airport)
        except RequestException as e:
            retries += 1
            print(f"Error: {e}, retrying ({retries}/{max_retries})...")
            # time.sleep(delay)
    return None, None, None, None, None  # If all retries fail, return None values

batch_size = 2000  # Smaller batch size
num_batches = len(data_LSGG_reduced) // batch_size + 1

for i in tqdm.tqdm(range(num_batches)):
    if not os.path.exists(os.path.join(file_path + f"landing_df_LSGG_with_meteo_{i}_of_{num_batches}.parquet")):
        batch = data_LSGG_reduced.iloc[i * batch_size:(i + 1) * batch_size]
        batch[["avg_wind_dir", "avg_wind_speed", "avg_vis", "avg_temp", "avg_press"]] = batch.apply(
            lambda row: pd.Series(get_meteo_data_with_retry(row, "LSGG")), axis=1
        )
        batch.to_parquet(os.path.join(file_path + f"landing_df_LSGG_with_meteo_{i}_of_{num_batches}.parquet"))
    else:
        print("file already exists")
    # time.sleep(10)



c:\Users\kruu\git_folder\STAR_shortcut_OSN_paper\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/22 [00:00<?, ?it/s]

file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists
file already exists


C:\Users\kruu\AppData\Local\Temp\ipykernel_14244\3826286800.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch[["avg_wind_dir", "avg_wind_speed", "avg_vis", "avg_temp", "avg_press"]] = batch.apply(
C:\Users\kruu\AppData\Local\Temp\ipykernel_14244\3826286800.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch[["avg_wind_dir", "avg_wind_speed", "avg_vis", "avg_temp", "avg_press"]] = batch.apply(
C:\Users\kruu\AppData\Local\Temp\ipykernel_14244\3826286800.py:49: SettingWithCopyWarning: 
A value 